## PROYECTO INTEGRADOR


# **Avance 4. Modelos alternativos**

Jiram Cesar Villalpando Guerrero        A01793579

Josep Romagosa Llordén                  A01374637

José Francisco Muñoz Del Angel          A01794174





**3.1 Establecer las medidas de calidad del modelo de aprendizaje automático**

**3.2 Proporcionar un marco de referencia para evaluar y mejorar modelos más avanzados.**



In [25]:
import warnings
warnings.filterwarnings('ignore')

In [26]:
from google.colab import drive
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
import dask.dataframe as dd
import warnings
import pandas as pd
import time
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import RidgeCV, ElasticNetCV, LassoCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from joblib import Parallel, delayed

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
file_path = '/content/drive/MyDrive/data_source/'


csv_files = ['data2.csv']  # Solo dos archivos

# Leer archivos CSV y combinarlos en un DataFrame
dfs = [pd.read_csv(file_path + file) for file in csv_files]
data = pd.concat(dfs, ignore_index=True)

In [29]:
df = data[['CourierCode', 'Parcel', 'ZipCodeFrom', 'ZipCodeTo', 'TransitAt', 'DeliveredAt']]

In [30]:
df['ParcelDict'] = df['Parcel'].apply(json.loads)
df['Weight'] = df['ParcelDict'].apply(lambda x: x.get('Weight', 0))
df['Height'] = df['ParcelDict'].apply(lambda x: x.get('Height', 0))
df['Length'] = df['ParcelDict'].apply(lambda x: x.get('Length', 0))
df['Width'] = df['ParcelDict'].apply(lambda x: x.get('Width', 0))
df.drop(['Parcel', 'ParcelDict'], axis=1, inplace=True)

In [31]:
# Convertir columnas de fecha
df['TransitAt'] = pd.to_datetime(df['TransitAt'])
df['DeliveredAt'] = pd.to_datetime(df['DeliveredAt'])

In [32]:
# Calculo de tiempo en transito
df['TransitTime'] = (df['DeliveredAt'] - df['TransitAt']).dt.total_seconds() / 3600

In [33]:
# Transformación logaritmica de TransitTime para normalizar la distribucion
df['TransitTime_log'] = np.log1p(df['TransitTime'])

In [34]:
# Añadir caracteristicas basadas en la fecha
df['TransitAt_year'] = df['TransitAt'].dt.year
df['TransitAt_month'] = df['TransitAt'].dt.month
df['TransitAt_weekday'] = df['TransitAt'].dt.weekday

In [35]:
# Codificación de las variables categoricas
encoder = OneHotEncoder(sparse=False)
courier_status_encoded = encoder.fit_transform(df[['CourierCode']])
courier_status_features = encoder.get_feature_names_out(['CourierCode'])
courier_status_df = pd.DataFrame(courier_status_encoded, columns=courier_status_features)

# Crear imputador que reemplazará NaN con el valor más frecuente en la columna
imputer = SimpleImputer(strategy='most_frequent')

# Aplicar imputador a las columnas de mes y día de la semana
df['TransitAt_month'] = imputer.fit_transform(df[['TransitAt_month']])
df['TransitAt_weekday'] = imputer.fit_transform(df[['TransitAt_weekday']])

month_weekday_encoded = encoder.fit_transform(df[['TransitAt_month', 'TransitAt_weekday']])
month_weekday_features = encoder.get_feature_names_out(['TransitAt_month', 'TransitAt_weekday'])
month_weekday_df = pd.DataFrame(month_weekday_encoded, columns=month_weekday_features)

# Concatenar las nuevas caracteristicas codificadas al dataframe original
df_encoded = pd.concat([df, courier_status_df, month_weekday_df], axis=1)

In [36]:
# Eliminar columnas que ya no se necesitan
columns_to_drop = ['CourierCode', 'TransitAt', 'DeliveredAt', 'TransitAt_month', 'TransitAt_weekday']
df_encoded.drop(columns_to_drop, axis=1, inplace=True)

In [37]:
cols_to_check = ['ZipCodeTo', 'Weight', 'Height', 'Length', 'Width']
df_encoded[cols_to_check] = df_encoded[cols_to_check].applymap(lambda x: pd.to_numeric(x, errors='coerce'))

# Eliminar filas con NaN en las columnas especificadas
df_encoded = df_encoded.dropna(subset=cols_to_check)

In [38]:
# Imputar valores nan
imputer = SimpleImputer(strategy='mean')
df_encoded_filled = imputer.fit_transform(df_encoded)
df_encoded_filled = pd.DataFrame(df_encoded_filled, columns=df_encoded.columns)

In [39]:
# Escalar las caracteristicas numericas
scaler = StandardScaler()
df_encoded_scaled = scaler.fit_transform(df_encoded_filled)

In [40]:
# Selección de caracteristicas mediante umbral de varianza
selector = VarianceThreshold(threshold=0.95)
df_reduced = selector.fit_transform(df_encoded_scaled)



---


# Entrega 4


3.3 Explorar una gama diversa de técnicas y enfoques con el fin de identificar el de mejor desempeño en el conjunto de datos en cuestión.

3.4 Encontrar la configuración óptima que maximiza el rendimiento del modelo en una tarea específica.

In [41]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LassoCV

In [42]:
features = ['ZipCodeFrom', 'ZipCodeTo'] + list(courier_status_features) + list(month_weekday_features)
X = df_encoded_filled[features]
y = df_encoded_filled['TransitTime']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
# Definir las características y el objetivo
features = ['ZipCodeFrom', 'ZipCodeTo'] + list(courier_status_features) + list(month_weekday_features)
X = df_encoded_filled[features]
y = df_encoded_filled['TransitTime']

# Reducir el tamaño del conjunto de datos para pruebas rápidas
sample_size = 10000
X_sample, _, y_sample, _ = train_test_split(X, y, train_size=sample_size, random_state=42)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

# Definir la función para entrenar y evaluar un modelo
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    return mse, r2, mae, training_time

# Definir los modelos con parámetros ajustados
models = {
    'SVR': SVR(C=1.0, epsilon=0.2),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
    'KNN': KNeighborsRegressor(n_neighbors=5),
    'Ridge Regression': RidgeCV(alphas=np.logspace(-3, 3, 7)),
    'ElasticNet': ElasticNetCV(cv=3, random_state=42),
    'Lasso': LassoCV(cv=3, random_state=42)
}

# Evaluar modelos en paralelo
def evaluate(name, model):
    mse, r2, mae, training_time = train_and_evaluate_model(model, X_train, y_train, X_test, y_test)
    return name, {'MSE': mse, 'R2 Score': r2, 'MAE': mae, 'Training Time': training_time}

# Realizar evaluación de modelos en paralelo
results = Parallel(n_jobs=-1)(delayed(evaluate)(name, model) for name, model in models.items())
results = dict(results)

# Imprimir los resultados
for model_name, metrics in results.items():
    print(f"\nModel: {model_name}")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.4f}")





Model: SVR
MSE: 3459.6735
R2 Score: -0.0226
MAE: 38.3801
Training Time: 13.2912

Model: Gradient Boosting
MSE: 2477.1551
R2 Score: 0.2678
MAE: 30.7120
Training Time: 2.7625

Model: KNN
MSE: 3402.8580
R2 Score: -0.0058
MAE: 38.5181
Training Time: 0.0051

Model: Ridge Regression
MSE: 2851.8275
R2 Score: 0.1570
MAE: 34.5772
Training Time: 0.0643

Model: ElasticNet
MSE: 3375.0645
R2 Score: 0.0024
MAE: 38.8269
Training Time: 0.2743

Model: Lasso
MSE: 3375.0648
R2 Score: 0.0024
MAE: 38.8269
Training Time: 0.5128


In [49]:
# Seleccionar los dos mejores modelos basado en R2 Score
sorted_results = sorted(results.items(), key=lambda item: item[1]['R2 Score'], reverse=True)
best_models = sorted_results[:2]

# Imprimir los mejores modelos
print("\nBest Models:")
for model_name, metrics in best_models:
    print(f"\nModel: {model_name}")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.4f}")

# Definir los parámetros de búsqueda para los dos mejores modelos
param_grids = {
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.1, 0.05],
        'max_depth': [3, 4, 5]
    },
    'ElasticNet': {
        'l1_ratio': [0.1, 0.5, 0.9],
        'alphas': [0.01, 0.1, 1.0, 10.0]
    }
}

best_params = {}


Best Models:

Model: Gradient Boosting
MSE: 2477.1551
R2 Score: 0.2678
MAE: 30.7120
Training Time: 2.7625

Model: Ridge Regression
MSE: 2851.8275
R2 Score: 0.1570
MAE: 34.5772
Training Time: 0.0643


In [50]:

# Ajustar los hiperparámetros de los mejores modelos
for model_name, _ in best_models:
    if model_name in param_grids:
        param_grid = param_grids[model_name]
        grid_search = GridSearchCV(models[model_name], param_grid, cv=3, n_jobs=-1, scoring='r2')
        grid_search.fit(X_train, y_train)
        best_params[model_name] = grid_search.best_params_
        models[model_name] = grid_search.best_estimator_

# mejores parámetros
print("\nBest Parameters:")
for model_name, params in best_params.items():
    print(f"{model_name}: {params}")





Best Parameters:
Gradient Boosting: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}


In [46]:
# Reevaluar los modelos con los mejores hiperparámetros
final_results = {}
for name, _ in best_models:
    mse, r2, mae, _ = train_and_evaluate_model(models[name], X_train, y_train, X_test, y_test)
    final_results[name] = {'MSE': mse, 'R2 Score': r2, 'MAE': mae}

# Imprimir los resultados finales
print("\nFinal Results:")
for model_name, metrics in final_results.items():
    print(f"\nModel: {model_name}")


Final Results:

Model: Gradient Boosting

Model: Ridge Regression


# **Análisis y Conclusiones**


SVR (Support Vector Regression)

MSE: 3459.6735
R2 Score: -0.0226
MAE: 38.3801
Training Time: 13.1048
El modelo SVR muestra un rendimiento negativo con un R² por debajo de 0, indicando que el modelo es peor que un modelo de referencia que predice el valor medio. El MSE y MAE son altos, y el tiempo de entrenamiento es considerablemente largo.

Gradient Boosting

MSE: 2477.1551
R2 Score: 0.2678
MAE: 30.7120
Training Time: 5.6991
Gradient Boosting muestra el mejor rendimiento entre todos los modelos iniciales, con un R² positivo y el más alto (0.2678), lo que indica que puede explicar aproximadamente el 26.78% de la variabilidad en los datos. El MSE y MAE son más bajos en comparación con otros modelos y el tiempo de entrenamiento es razonable.

KNN (K-Nearest Neighbors)

MSE: 3402.8580
R2 Score: -0.0058
MAE: 38.5181
Training Time: 0.0169
KNN también muestra un rendimiento negativo en términos de R², similar a SVR, con MSE y MAE altos. Sin embargo, el tiempo de entrenamiento es extremadamente corto.

Ridge Regression

MSE: 2851.8275
R2 Score: 0.1570
MAE: 34.5772
Training Time: 0.2772
Ridge Regression tiene un rendimiento decente con un R² positivo (0.1570), aunque no tan bueno como Gradient Boosting. Muestra un MSE y MAE moderados y un tiempo de entrenamiento corto.

ElasticNet

MSE: 3375.0645
R2 Score: 0.0024
MAE: 38.8269
Training Time: 0.6554
ElasticNet muestra un rendimiento similar al de Lasso, con un R² cercano a 0, indicando que casi no tiene poder predictivo. El MSE y MAE son altos y el tiempo de entrenamiento es bajo a moderado.

Lasso

MSE: 3375.0648
R2 Score: 0.0024
MAE: 38.8269
Training Time: 0.4149
Lasso muestra un rendimiento casi idéntico al de ElasticNet, con los mismos valores de MSE, R² y MAE, y un tiempo de entrenamiento ligeramente más corto.



# Conclusiones

Los resultados de los modelos muestran que Gradient Boosting y Ridge Regression son las opciones más prometedoras. Gradient Boosting tuvo el mejor rendimiento, con un MSE de 2477.1551, R² de 0.2678 y MAE de 30.7120, demostrando ser el modelo más preciso en explicar la variabilidad en los datos. Aunque el tiempo de entrenamiento fue de 5.6991 segundos, su capacidad predictiva justifica esta inversión de tiempo. Ridge Regression también tuvo un rendimiento decente, con un MSE de 2851.8275, R² de 0.1570 y MAE de 34.5772, destacándose por su velocidad de entrenamiento de 0.2772 segundos.



En contraste, los modelos SVR, KNN, ElasticNet y Lasso mostraron un desempeño insatisfactorio, con altos valores de MSE y MAE y R² negativos o cercanos a cero. Esto indica una baja capacidad predictiva, especialmente en comparación con Gradient Boosting y Ridge Regression. En conclusión, se recomienda utilizar Gradient Boosting como el modelo final debido a su mayor precisión, optimizado con los mejores hiperparámetros {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}, mientras que Ridge Regression puede considerarse una alternativa rápida y razonablemente precisa.






